# Array Displays

Like `CameraDisplays`, ctapipe provides a way to display information related to the array on the ground: `ArrayDisplay`

In [ ]:
from ctapipe.visualization import ArrayDisplay
from ctapipe.instrument import SubarrayDescription
from ctapipe.coordinates import GroundFrame, EastingNorthingFrame
from ctapipe.containers import HillasParametersContainer

from astropy.coordinates import SkyCoord
from astropy import units as u

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (8, 6)

In [ ]:
tel_ids = list(range(1, 5)) + list(range(5, 20))  # just LSTs  + one set of MSTs

subarray = SubarrayDescription.read(
    "dataset://gamma_20deg_0deg_run1___cta-prod5-lapalma_desert-2158m-LaPalma-dark_100evts.simtel.zst"
).select_subarray(tel_ids)

An array display is created for example in `subarray.peek()`:

In [ ]:
subarray.peek()

However, you can make one manually with a bit more flexibility:

## Constructing an ArrayDisplay

In [ ]:
disp = ArrayDisplay(subarray)

You can specify the Frame you want as long as it is compatible with `GroundFrame`.  `EastingNorthingFrame` is probably the most useful.  You can also add telescope labels

In [ ]:
disp = ArrayDisplay(subarray, frame=EastingNorthingFrame())
disp.add_labels()

## Using color to show information 

By default the color  of the telescope circles correlates to telescope type. However, you can use color to convey other information  by setting the `values` attribute, like a trigger pattern

In [ ]:
plt.set_cmap("rainbow")  # the array display will use the current colormap for values

ad = ArrayDisplay(subarray)
ad.telescopes.set_linewidth(0)  # to turn off the telescope borders

trigger_pattern = np.zeros(subarray.n_tels)
trigger_pattern[
    [
        1,
        4,
        5,
        6,
    ]
] = 1
ad.values = trigger_pattern  # display certain telescopes in a color
ad.add_labels()

or for example, you could use color to represent the telescope distance to the impact point

In [ ]:
shower_impact = c.SkyCoord(200 * u.m, -200 * u.m, 0 * u.m, frame=EastingNorthingFrame())

In [ ]:
plt.set_cmap("rainbow")  # the array display will use the current colormap for values
ad = ArrayDisplay(subarray)
ad.telescopes.set_linewidth(0)  # to turn off the telescope borders
plt.scatter(shower_impact.easting, shower_impact.northing, marker="+", s=200)

distances = np.hypot(
    subarray.tel_coords.cartesian.x - shower_impact.cartesian.x,
    subarray.tel_coords.cartesian.y - shower_impact.cartesian.y,
)
ad.values = distances
plt.colorbar(ad.telescopes, label="Distance (m)")

## Overlaying vectors

For plotting reconstruction quantities, it's useful to overlay vectors on the telescope positions. `ArrayDisplay` provides functions:
* `set_vector_uv` to set by cartesian coordinates from the center of each telescope
* `set_vector_rho_phi` to set by polar coorinates from the center of each telescope
* `set_vector_hillas` to set vectors from a `dict[int,HillasParameters]` mapping tel_id (not index!) to a set of parameters. 

In [ ]:
np.random.seed(0)
phis = np.random.uniform(0, 180.0, size=subarray.n_tels) * u.deg
rhos = np.ones(subarray.n_tels) * 200 * u.m


ad = ArrayDisplay(subarray, frame=EastingNorthingFrame(), tel_scale=2)
ad.set_vector_rho_phi(rho=rhos, phi=phis)